# AI Ontology Graph Visualizer

This notebook loads `ontology.json`, builds a directed graph of the AI supply chain ontology, and visualizes it.

Usage:
- Run the install cell (only needed once per environment) to ensure `networkx` is available.
- Run the main visualization cell.
- Adjust `selected_layers` in the code to view specific layers (e.g., `[2]` for Layer 2, `[1,2,3]` for multiple layers).

In [ ]:
# Install networkx if not present (safe to re-run)
try:
    import networkx as nx
except ImportError:
    !pip install networkx
    import networkx as nx

import json
import matplotlib.pyplot as plt

In [ ]:
# === CONFIGURATION ===
# Change this list to control which layers are visualized.
# Example: [2] for only Layer 2, [1,2,3] for Layers 1-3, [] for all layers.
selected_layers = [2]  # <-- edit this as needed

# Node size scaling
BASE_NODE_SIZE = 800

# Edge arrow style
ARROW_STYLE = '-|>'

# === LOAD ONTOLOGY ===
with open('../ontology.json', 'r') as f:
    data = json.load(f)

nodes = data.get('nodes', [])
edges = data.get('edges', [])

# === BUILD GRAPH ===
G = nx.DiGraph()

for n in nodes:
    node_id = n['id']
    G.add_node(node_id, **n)

for e in edges:
    src = e['source']
    tgt = e['target']
    G.add_edge(src, tgt, **e)

# === FILTER BY LAYER (if selected_layers not empty) ===
if selected_layers:
    filtered_nodes = [n for n, d in G.nodes(data=True) if d.get('layer') in selected_layers]
    H = G.subgraph(filtered_nodes).copy()
else:
    H = G.copy()

if len(H.nodes) == 0:
    print('No nodes to display for selected_layers =', selected_layers)
else:
    # === LAYER → COLOR MAPPING ===
    # Use a simple colormap for layers 1-7
    import math
    cmap = plt.cm.get_cmap('tab10')

    def layer_to_color(layer):
        if layer is None:
            return (0.5, 0.5, 0.5, 1.0)  # gray fallback
        # Map layer 1-7 into 0-1 range of tab10
        idx = (layer - 1) % 10
        return cmap(idx)

    node_colors = []
    node_sizes = []
    labels = {}

    for n, d in H.nodes(data=True):
        layer = d.get('layer')
        node_colors.append(layer_to_color(layer))
        node_sizes.append(BASE_NODE_SIZE)
        labels[n] = d.get('name', n)

    # === LAYOUT ===
    # Spring layout is a good default; seed for reproducibility
    pos = nx.spring_layout(H, k=0.8, seed=42)

    # === DRAW ===
    plt.figure(figsize=(12, 8))
    nx.draw_networkx_nodes(H, pos,
                           node_color=node_colors,
                           node_size=node_sizes,
                           alpha=0.9)

    # Edge styling by type
    dep_edges = [(u, v) for u, v, d in H.edges(data=True) if d.get('type') == 'dependency']
    comp_edges = [(u, v) for u, v, d in H.edges(data=True) if d.get('type') == 'competition']
    lev_edges = [(u, v) for u, v, d in H.edges(data=True) if d.get('type') == 'leverage']

    nx.draw_networkx_edges(H, pos, edgelist=dep_edges, arrows=True, arrowstyle=ARROW_STYLE, alpha=0.6)
    nx.draw_networkx_edges(H, pos, edgelist=lev_edges, arrows=True, arrowstyle=ARROW_STYLE, style='dashed', alpha=0.6)
    # Competition: undirected feel (draw without arrows)
    nx.draw_networkx_edges(H, pos, edgelist=comp_edges, arrows=False, style='dotted', alpha=0.6)

    nx.draw_networkx_labels(H, pos, labels, font_size=9)

    plt.title(f"AI Supply Chain Ontology — Layers {selected_layers if selected_layers else 'ALL'}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    # === LEGEND FOR LAYERS ===
    # Build a small legend for layer colors actually present in H
    present_layers = sorted({d.get('layer') for _, d in H.nodes(data=True) if d.get('layer') is not None})
    if present_layers:
        print("Layers present in this view:")
        for L in present_layers:
            print(f"  Layer {L}")